# JVN-InvenRecon-bulkFormat

### Author: Ekin Yelken
Developed: May 2021

Client: Self

### Description: 
The objective of this code is to bulk format the data from an inventory reconciliation form (variable 'file') into an output csv (variable 'out) so it may be easily copy pasted into the 'JVN Inventory - v2' file.

The input must be formated as a UTF-8 comma separrated value file. This can be done by using the "Save As" function in Excel and changing the file type in the drop down menu to '.csv'.

### Work flow:
- Import libraries
- Determine I/O files
- Read In file and write data to 'rows' array
- Find Number of locations and their coressponding row
- Find Row where titles are located and their coressponding column
- Get Data:
    - Open output file and write names of parameters
    - Running L times for number of locations, find the start and end of the technologies entries for location N
    - Calculate values of interest
    - If entry is valid (not related to cables, floor track, cabling, or is not a misc/shiping/ delivery expense, add row to output
- Close and reopen Output file (if already open or just open if not already open) to see results

## Import Libraries

Import necessary libraries, declate 'rows' array holding input data, declate file I/O names

In [1]:
import csv
import numpy as np
file = 'MAG-463-13-440 Kent.csv' # name of Reconciliation file
out = 'Output.csv'
rows = [] # array of all of the rows in the input csv

## Open Data

Open Inventory Reconciliation File and Get Row data

In [2]:
with open(file, 'r') as my_file:
    reader = csv.reader(my_file)
    rows = list(reader)

## Location Names

Return part of a string. In a cell containing "Location:XXXXX", this function returns only "XXXXX"

In [3]:
def getLoc(STR):
    return STR[10:len(STR)]

## Location Quanitity

Number of locations in room may vary, this counts the number of locations

In [4]:
L=0
length = len(rows)
for N in range(length):
    STR = rows[N][0]
    wordLoc = STR[0:8]
    if(wordLoc == 'Location'):
        L+=1

## Find Location Row Number

With number of locations, set an array called anchor which are the "anchor" row numbers

In [5]:
anchor = np.zeros(L+1)
counter = 0
r = 7

for N in range(length):
    if (counter == L): 
        anchor[-1] = len(rows)
        break
    STR = rows[N][0]
    wordLoc = STR[0:8]
    Loc = getLoc(STR)
    if(wordLoc == 'Location'):
        anchor[counter] = N
        counter+=1
    if(STR == "MANUFACTURER"):
        r = N

## Parameter Coumn Number

'r' symbolizes the row that the variable names are on. This number is based on formatting as when saved to CSV, the line containing parameter names is consistently 2 rows above the first "Location:XXXXX"

In [6]:
p = 0
for index in range(13):
    if(rows[r][index][0:3] == "MAN"):
        #print("manu index found", index)
        manufacturer = index
    elif(rows[r][index][0:3] == "MOD"):
        #print("model index found", index)
        model = index
    elif(rows[r][index][0:11] == "DESCRIPTION"):
        #print("desc index found", index)
        description = index
    elif(rows[r][index][0:3] == "SER"):
        #print("seri index found", index)
        serial = index
    elif(rows[r][index][0:3] == "ORI"):
        #print("quant index found", index)
        quantity = index
    elif(rows[r][index][0:3] == "INS"):
        #print("inst index found", index)
        install = index
    elif(rows[r][index][0:3] == "PAR"):
        p+=1
    elif(rows[r][index][0:3] == ""):
        p+=1

## Set Variable Names from CSV

Set string names for referencing

In [7]:
MANF = rows[r][manufacturer]
MODEL = rows[r][model]
DESC = rows[r][description]
SERIAL = rows[r][serial]
QTY = rows[r][quantity]
LOCATION = "LOCATION"

## Array of Variable Names

In [8]:
names = []
names.append(str(MANF))
names.append(str(MODEL))
names.append(str('PART NO.'))
names.append(str(DESC))
names.append(str(QTY))
names.append(str(SERIAL))
names.append(str('INSALLED ACCESORIES'))
names.append(str(LOCATION.upper()))
#print(names)
#print("column numbers")
#print(manufacturer, model, description, serial, quantity, install)

## Get Location & Set correct Number of entries

Format Data based on parameters

In [9]:
total = 0
subtracted = 0

with open(out, mode='w') as output:
    # open output file and write names of parameters
    writer = csv.writer(output, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(names)
    
    # This will run L times for number of locations
    for N in range(1,L+1):

        index = int(anchor[N-1]) # row index where technology for location N begins
        final = int(anchor[N]) # row index where techbology for lcoation N ends

        numTech = final-index-1 # number of technologies at location N
        locationName = getLoc(str(rows[index][0])) # name of the location
        total += numTech # total number of entries
        
        # ignore any cable related entires
        if (locationName != 'Floor Track' and locationName != 'Cable' and locationName != 'Cabling'):
            for i in range(numTech):
                Idx = index+i+1 # index number
                if (rows[Idx][0] != ""):
                    # write a row into output file with correct data
                    writer.writerow(np.array([rows[Idx][manufacturer], rows[Idx][model], "", rows[Idx][description], rows[Idx][quantity][0:1], rows[Idx][serial], rows[Idx][install][0:1], str(locationName)]))
                else: # if manufacturer is blank, entry is most probably some misc/shiping/ delivery expense, ignore it
                    subtracted += 1
        else: # calculate final number of entries made accounting for entries ignored.
            subtracted += numTech

print("Done!")
#print("Final Number Entries: ", total-subtracted)

Done!
